## Predict historical prices of Brent Oil, Crude Oil WTI, Natural Gas, Heating Oil from 2000-2022 with Fedot.Industrial

Dataset published on Kaggle3 consists of historical prices of Brent Oil, CrudeOil WTI, Natural Gas, and Heating Oil from 2000 to 2022. This sample of DailyOilGasPrices was created by using 30 consecutive business days of **Crude Oil WTI close prices** and **traded volumes** as **predictors** and the **average natural gas close** price during each 30-day time frame as the **target** variable. The final dataset has 191 2-dimensional time series of length 30, of which 70% were randomly sampled as training data and the remaining 30% as testing data. This type of model could help companies and governments to better analyse and predict economic situations and correlations regarding oil and natural gas.
Link to the dataset - https://www.kaggl.com/datasets/prasertk/historical-daily-oil-and-natural-gas-prices

In [1]:
import pandas as pd

from fedot_ind.api.utils.path_lib import PROJECT_PATH
from fedot.core.pipelines.pipeline_builder import PipelineBuilder
from fedot_ind.tools.loader import DataLoader
from fedot_ind.core.repository.initializer_industrial_models import IndustrialModels
from examples.example_utils import init_input_data, calculate_regression_metric
from fedot.core.pipelines.tuning.tuner_builder import TunerBuilder
from fedot.core.repository.metrics_repository import RegressionMetricsEnum
from fedot_ind.core.optimizer.IndustrialEvoOptimizer import IndustrialEvoOptimizer
from golem.core.tuning.simultaneous import SimultaneousTuner
import matplotlib
from fedot_ind.api.main import FedotIndustrial
matplotlib.use('TkAgg')

The list of basic fedot industrial models for experiment are shown below. We using simple linear machine learning pipelines with 3 different feature generators: Statistical, Reccurence, Topological. And for each of them we add PCA transformation with 90 % of explained dispersion.

In [2]:
model_dict = {
    'regression_with_statistical_features': PipelineBuilder().add_node('quantile_extractor',
                                                                       params={'window_size': 5}).add_node('ridge')
}
experiment_setup = {'problem': 'regression',
                    'metric': 'rmse',
                    'timeout': 10,
                    'num_of_generations': 5,
                    'pop_size': 10,
                    'logging_level': 40,
                    'available_operations':
                        ['rfr',
                         'ridge',
                         'gbr',
                         'sgdr',
                         'linear',
                         'xgbreg',
                         'dtreg',
                         'treg',
                         'knnreg',
                         'scaling',
                         'normalization',
                         'pca'
                         'eigen_basis',
                         'fourier_basis',
                         'minirocket_extractor',
                         'quantile_extractor',
                         'signal_extractor'
                         ],
                    'n_jobs': 4,
                    'industrial_preprocessing': True,
                    'initial_assumption': None,
                    'max_pipeline_fit_time': 5,
                    'with_tuning': False,
                    'early_stopping_iterations': 5,
                    'early_stopping_timeout': 10,
                    'optimizer': IndustrialEvoOptimizer}

dataset_name = 'DailyOilGasPrices'
data_path = PROJECT_PATH + '/examples/data'
experiment_setup['output_folder'] = f'./{dataset_name}/results_of_experiment'
OperationTypesRepository = IndustrialModels().setup_repository()
tuning_params = {'task': 'regression',
                 'metric': RegressionMetricsEnum.RMSE,
                 'tuning_timeout':10,
                 'tuning_iterations':30}

In [17]:
def evaluate_industrial_model(train_data, test_data, task:str = 'regression'):
    metric_dict = {}
    input_data = init_input_data(train_data[0], train_data[1], task=task)
    val_data = init_input_data(test_data[0], test_data[1], task=task)
    for model in model_dict.keys():
        print(f'Current_model - {model}')
        pipeline = model_dict[model].build()
        pipeline.fit(input_data)
        features = pipeline.predict(val_data).predict
        metric = calculate_regression_metric(test_target=test_data[1], labels=features)
        metric_dict.update({model: metric})
    return metric_dict

def tuning_industrial_pipelines(pipeline, tuning_params, train_data):
    input_data = init_input_data(train_data[0], train_data[1], task=tuning_params['task'])
    tuning_method = SimultaneousTuner
    pipeline_tuner = TunerBuilder(input_data.task) \
        .with_tuner(tuning_method) \
        .with_metric(tuning_params['metric']) \
        .with_timeout(tuning_params['tuning_timeout']) \
        .with_iterations(tuning_params['tuning_iterations']) \
        .build(input_data)

    pipeline = pipeline_tuner.tune(pipeline)
    return pipeline

def evaluate_automl(runs = 5):
    metric_dict = {}
    model_list = []

    if 'tuning_params' in experiment_setup.keys():
        del experiment_setup['tuning_params']

    if 'industrial_preprocessing' in experiment_setup.keys():
        ind_preproc = experiment_setup['industrial_preprocessing']
        del experiment_setup['industrial_preprocessing']
    else:
        ind_preproc = True

    for run in range(runs):
        model = FedotIndustrial(**experiment_setup)
        model.preprocessing = ind_preproc
        model.fit(train_data)
        prediction = model.predict(test_data)

        metric = calculate_regression_metric(test_target=test_data[1], labels=prediction)
        metric = metric.T
        metric.columns = metric.columns.values

        metric_dict.update({f'run_number - {run}':metric})
        model_list.append(model)

    return metric_dict, model_list

Now we must download the dataset. It could be done by using `DataReader` class that implemented as attribute of `FedotIndustrial` class. This class firstly tries to read the data from local project folder `data_path` and then if it is not possible, it downloads the data from the UCR/UEA archive. The data will be saved in the `data` folder.

In [4]:
_, train_data, test_data = DataLoader(dataset_name=dataset_name).read_train_test_files(
            dataset_name=dataset_name,
            data_path=data_path)

2024-01-11 17:08:08,298 - Reading data from D:\WORK\Repo\Industiral\IndustrialTS/examples/data/DailyOilGasPrices


In [5]:
input_data = init_input_data(train_data[0], train_data[1], task=tuning_params['task'])
val_data = init_input_data(test_data[0], test_data[1], task=tuning_params['task'])

Lets check our data.

In [6]:
input_data.features.shape

(133, 2, 30)

Lets visualise our predictors.

In [7]:
from matplotlib import pyplot as plt
pd.DataFrame(input_data.features[1, 0, :]).plot(title='Crude Oil WTI close prices')
pd.DataFrame(input_data.features[1, 1, :]).plot(title='Crude Oil WTI traded volumes')
plt.show()

Next steps are quite straightforward. We need to fit the model and then predict the values for the test data just like for any other model in sklearn.

At the `fit` stage FedotIndustrial will transform initial time series data into features dataframe and will train regression model.

In [8]:
metric_dict = evaluate_industrial_model(train_data,test_data)

Current_model - regression_with_statistical_features


100%|██████████| 58/58 [00:00<00:00, 796.64it/s]


At the end of the experiment we can obtain the desired metric values using `calculate_regression_metric` method. Now there are five available metrics for classification task:
- `explained_variance_score`
- `max_error`
- `mean_absolute_error`
- `mean_squared_error`
- `d2_absolute_error_score`.
- `median_absolute_error`
- `r2_score`

In [9]:
df_baseline = pd.concat([x for x in metric_dict.values()],axis=1)
df_baseline.columns = list(metric_dict.keys())
df_baseline = df_baseline.T
df_baseline

,r2_score:,mean_squared_error:,root_mean_squared_error:,mean_absolute_error,median_absolute_error,explained_variance_score,max_error,d2_absolute_error_score
regression_with_statistical_features,-5.644074,21.505536,4.637406,3.231781,2.051471,-5.295417,18.537842,-1.248095


In [10]:
best_baseline = df_baseline.sort_values('root_mean_squared_error:')['root_mean_squared_error:'].iloc[0]

## Could it be done better? Tuning approach

In [11]:
metric_dict = {}
for model in model_dict.keys():
    print(f'Current_model - {model}')
    pipeline = model_dict[model].build()
    tuned_pipeline = tuning_industrial_pipelines(pipeline,tuning_params,train_data)
    tuned_pipeline.fit(input_data)
    features = tuned_pipeline.predict(val_data).predict
    metric = calculate_regression_metric(test_target=test_data[1], labels=features)
    metric = metric.T
    metric.columns = metric.columns.values
    metric['model_params'] = metric['model_params'] = str({node:node.parameters for node in tuned_pipeline.graph_description['nodes']})
    metric_dict.update({model: metric})

Current_model - regression_with_statistical_features
2024-01-11 17:08:37,555 - DataSourceSplitter - Stratificated splitting of data is disabled.
2024-01-11 17:08:37,555 - DataSourceSplitter - Hold out validation is applied.
2024-01-11 17:08:37,558 - SimultaneousTuner - Hyperparameters optimization start: estimation of metric for initial graph


100%|██████████| 27/27 [00:00<00:00, 731.68it/s]


2024-01-11 17:08:38,009 - SimultaneousTuner - Initial graph: {'depth': 2, 'length': 2, 'nodes': [ridge, quantile_extractor]}
ridge - {}
quantile_extractor - {'window_size': 5} 
Initial metric: [7.546]
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]2024-01-11 17:08:38,014 - build_posterior_wrapper took 0.001994 seconds
2024-01-11 17:08:38,014 - TPE using 0 trials


  0%|          | 0/106 [00:00<?, ?it/s]

 57%|#####6    | 60/106 [00:00<00:00, 567.55it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

100%|##########| 27/27 [00:00<00:00, 731.69it/s]


 33%|███▎      | 1/3 [00:00<00:00,  2.21trial/s, best loss: 7.546356685867857]2024-01-11 17:08:38,467 - build_posterior_wrapper took 0.000997 seconds
2024-01-11 17:08:38,468 - TPE using 1/1 trials with best loss 7.546357


...

  0%|          | 0/106 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

100%|##########| 27/27 [00:00<00:00, 1353.58it/s]


100%|██████████| 30/30 [00:10<00:00,  2.63trial/s, best loss: 2.740364806635174]
2024-01-11 17:08:49,675 - SimultaneousTuner - Hyperparameters optimization finished


100%|██████████| 27/27 [00:00<00:00, 1289.14it/s]

2024-01-11 17:08:49,938 - SimultaneousTuner - Return tuned graph due to the fact that obtained metric 2.740 equal or better than initial (+ 0.05% deviation) 7.543
2024-01-11 17:08:49,939 - SimultaneousTuner - Final graph: {'depth': 2, 'length': 2, 'nodes': [ridge, quantile_extractor]}
ridge - {'alpha': 2.1380306940047142}
quantile_extractor - {'window_size': 0, 'stride': 1}
2024-01-11 17:08:49,939 - SimultaneousTuner - Final metric: 2.740



100%|██████████| 58/58 [00:00<00:00, 1530.38it/s]


In [13]:
df_tuned = pd.concat([x for x in metric_dict.values()],axis=0).T
df_tuned.columns = list(metric_dict.keys())
df_tuned.T.sort_values('root_mean_squared_error:')

,r2_score:,mean_squared_error:,root_mean_squared_error:,mean_absolute_error,median_absolute_error,explained_variance_score,max_error,d2_absolute_error_score,model_params
regression_with_statistical_features,0.426662,1.855782,1.362271,1.056372,0.880141,0.432458,4.171815,0.265165,"{ridge: {'alpha': 2.1380306940047142}, quantile_extractor: {'window_size': 0, 'stride': 1}}"


In [14]:
df_tuned.T.sort_values('root_mean_squared_error:')['model_params'].iloc[0]

"{ridge: {'alpha': 2.1380306940047142}, quantile_extractor: {'window_size': 0, 'stride': 1}}"

In [15]:
best_tuned = df_tuned.T.sort_values('root_mean_squared_error:')['root_mean_squared_error:'].iloc[0]

## Even better? AutoML approach

In [18]:
metric_dict, model_list = evaluate_automl(runs=3)

2024-01-11 17:23:04,362 - AssumptionsHandler - Memory consumption for fitting of the initial pipeline in main session: current 0.3 MiB, max: 1.8 MiB
2024-01-11 17:23:04,364 - ApiComposer - Initial pipeline was fitted in 1.0 sec.
2024-01-11 17:23:04,365 - AssumptionsHandler - Preset was changed to best_quality due to fit time estimation for initial model.
2024-01-11 17:23:04,373 - ApiComposer - AutoML configured. Parameters tuning: False. Time limit: 10 min. Set of candidate models: ['rfr', 'ridge', 'gbr', 'sgdr', 'linear', 'xgbreg', 'dtreg', 'treg', 'knnreg', 'scaling', 'normalization', 'pcaeigen_basis', 'fourier_basis', 'minirocket_extractor', 'quantile_extractor', 'signal_extractor'].
2024-01-11 17:23:04,397 - ApiComposer - Pipeline composition started.


Generations:   0%|          | 0/5 [00:00<?, ?gen/s]

2024-01-11 17:23:08,018 - IndustrialDispatcher - 2 individuals out of 2 in previous population were evaluated successfully.



SVD estimation:  58%|█████▊    | 61/106 [00:00<00:00, 128.53it/s]

SVD estimation:  99%|█████████▉| 105/106 [00:00<00:00, 225.27it/s]

 34%|███▍      | 36/106 [00:00<00:00, 247.24it/s]
SVD estimation: 136it [00:00, 214.62it/s]                         

100%|██████████| 106/106 [00:00<00:00, 240.46it/s][A

SVD estimation: 159it [00:01, 150.49it/s]
SVD estimation: 212it [00:01, 183.96it/s]

100%|██████████| 27/27 [00:00<00:00, 27.10it/s]

100%|██████████| 106/106 [00:00<00:00, 304.54it/s][A

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

  6%|▌         | 6/106 [00:00<00:02, 42.01it/s]

 44%|████▍     | 12/27 [00:00<00:00, 24.29it/s]

 17%|█▋        | 18/106 [00:00<00:02, 31.72it/s]

 67%|██████▋   | 18/27 [00:00<00:00, 21.02it/s]

 25%|██▍       | 26/106 [00:00<00:03, 25.90it/s]

 89%|████████▉ | 24/27 [00:01<00:00, 17.61it/s]

100%|██████████| 27/27 [00:01<00:00, 16.53it/s]


 34%|███▍      | 36/106 [00:01<00:05, 13.38it/s]

100%|██████████| 106/106 [0

2024-01-11 17:25:06,035 - IndustrialDispatcher - 12 individuals out of 13 in previous population were evaluated successfully.


...

Generations:  40%|████      | 2/5 [09:02<13:24, 268.05s/gen]

2024-01-11 17:47:23,601 - GroupedCondition - Optimisation stopped: Time limit is reached


Generations:  40%|████      | 2/5 [09:02<13:33, 271.20s/gen]

2024-01-11 17:47:23,714 - ApiComposer - Model generation finished


2024-01-11 17:47:24,163 - FEDOT logger - Final pipeline was fitted
2024-01-11 17:47:24,165 - FEDOT logger - Final pipeline: {'depth': 1, 'length': 1, 'nodes': [treg]}
treg - {}
2024-01-11 17:47:24,166 - MemoryAnalytics - Memory consumption for finish in main session: current 1.0 MiB, max: 26.2 MiB


In [19]:
df_automl = pd.concat([x for x in metric_dict.values()],axis=0).T
df_automl.columns = list(metric_dict.keys())
df_automl = df_automl.T
df_automl.sort_values('root_mean_squared_error:')

,r2_score:,mean_squared_error:,root_mean_squared_error:,mean_absolute_error,median_absolute_error,explained_variance_score,max_error,d2_absolute_error_score
run_number - 0,0.607232,1.271311,1.127524,0.799539,0.640049,0.607232,4.497106,0.443824
run_number - 1,0.607232,1.271311,1.127524,0.799539,0.640049,0.607232,4.497106,0.443824
run_number - 2,0.607232,1.271311,1.127524,0.799539,0.640049,0.607232,4.497106,0.443824


In [22]:
for model in model_list:
    model.solver.current_pipeline.show()
    model.plot_operation_distribution(mode='each')
    model.plot_fitness_by_generation()

## Compare with State of Art (SOTA) models

In [49]:
df = pd.read_csv(data_path+'/ts_regression_sota_results.csv',sep=';')

In [50]:
df = df[df['ds/type'] == dataset_name].iloc[:,:25]
df.index = df['algorithm']
df = df.drop(['algorithm','ds/type'], axis=1)
df = df.replace(',','.', regex=True).astype(float)
df['Fedot_Industrial_baseline'] = best_baseline
df['Fedot_Industrial_tuned'] = best_tuned
df['Fedot_Industrial_AutoML'] = 0

In [51]:
df.loc['min','Fedot_Industrial_AutoML'] = df_automl['root_mean_squared_error:'].min()
df.loc['max','Fedot_Industrial_AutoML'] = df_automl['root_mean_squared_error:'].max()
df.loc['average','Fedot_Industrial_AutoML'] = df_automl['root_mean_squared_error:'].mean()
df = df.T

In [58]:
df.sort_values('min')['min']

FreshPRINCE_RMSE             1.020426
DrCIF_RMSE                   1.046399
RotF_RMSE                    1.097089
Fedot_Industrial_AutoML      1.127524
TSF_RMSE                     1.172730
RandF_RMSE                   1.182821
XGBoost_RMSE                 1.272832
RIST_RMSE                    1.301506
Fedot_Industrial_tuned       1.362271
RDST_RMSE                    1.382981
MultiROCKET_RMSE             1.454406
SingleInception_RMSE         1.490343
InceptionT_RMSE              1.542210
5NN-DTW_RMSE                 1.593567
CNN_RMSE                     1.602674
FPCR-Bs_RMSE                 1.607024
ResNet_RMSE                  1.610736
FCN_RMSE                     1.636335
FPCR_RMSE                    1.637966
Grid-SVR_RMSE                1.699988
Ridge_RMSE                   1.713667
ROCKET_RMSE                  1.856612
5NN-ED_RMSE                  1.905470
1NN-ED_RMSE                  2.129638
1NN-DTW_RMSE                 2.211915
Fedot_Industrial_baseline    4.637406
Name: min, d

In [57]:
df.sort_values('max')['max']

Fedot_Industrial_AutoML      1.127524
Fedot_Industrial_tuned       1.362271
RIST_RMSE                    1.887153
RotF_RMSE                    2.088333
FreshPRINCE_RMSE             2.101253
MultiROCKET_RMSE             2.137563
TSF_RMSE                     2.181236
RDST_RMSE                    2.200364
DrCIF_RMSE                   2.201153
ResNet_RMSE                  2.404590
RandF_RMSE                   2.446314
5NN-DTW_RMSE                 2.502464
XGBoost_RMSE                 2.511297
FPCR-Bs_RMSE                 2.512605
FPCR_RMSE                    2.523426
5NN-ED_RMSE                  2.554152
FCN_RMSE                     2.559661
InceptionT_RMSE              2.574168
SingleInception_RMSE         2.633115
CNN_RMSE                     2.674200
Grid-SVR_RMSE                2.800419
ROCKET_RMSE                  2.870444
Ridge_RMSE                   2.904762
1NN-ED_RMSE                  3.281798
1NN-DTW_RMSE                 3.408802
Fedot_Industrial_baseline    4.637406
Name: max, d

In [56]:
df.sort_values('average')['average']

Fedot_Industrial_AutoML      1.127524
Fedot_Industrial_tuned       1.362271
FreshPRINCE_RMSE             1.490442
RIST_RMSE                    1.501047
RotF_RMSE                    1.559385
DrCIF_RMSE                   1.594442
TSF_RMSE                     1.684828
RandF_RMSE                   1.708196
XGBoost_RMSE                 1.716903
RDST_RMSE                    1.772813
MultiROCKET_RMSE             1.773578
ResNet_RMSE                  1.938074
InceptionT_RMSE              2.030315
FPCR_RMSE                    2.052389
5NN-DTW_RMSE                 2.055256
FCN_RMSE                     2.069046
FPCR-Bs_RMSE                 2.097964
SingleInception_RMSE         2.149368
CNN_RMSE                     2.150854
Grid-SVR_RMSE                2.203537
5NN-ED_RMSE                  2.251424
ROCKET_RMSE                  2.275254
Ridge_RMSE                   2.363609
1NN-DTW_RMSE                 2.742105
1NN-ED_RMSE                  2.822595
Fedot_Industrial_baseline    4.637406
Name: averag